In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from prettytable import PrettyTable


In [3]:
def avg_diff(values):
    # return the average pairwise different between all the pairs in the list
    pairs = [(a, b) for idx, a in enumerate(values) for b in values[idx + 1:]]
    difference = list(map(lambda x: abs(x[0] - x[1]), pairs))
    return sum(difference)/len(difference)


In [9]:
# score_dir = '../bold/result/score'
# score_dir = 'results/dexperts_gpt2_med_alpha1/score'
score_dir = 'results/dexperts_gpt2_race_alpha2/score'
path_to_dir = os.path.relpath(score_dir)
# get the json files in the directory
csv_files = [pos_csv for pos_csv in os.listdir(path_to_dir) if pos_csv.endswith('.csv')]

df = pd.concat([pd.read_csv(os.path.join(path_to_dir, file))
               for file in csv_files])
df

,model,group,subgroup,metric,score
0,dexperts_gpt2,gpt2_race,Asian_Americans,regard-positive,0.6451
1,dexperts_gpt2,gpt2_race,Asian_Americans,regard-negative,0.1215
2,dexperts_gpt2,gpt2_race,Asian_Americans,regard-neutral,0.0479
3,dexperts_gpt2,gpt2_race,Asian_Americans,regard-other,0.1856
4,dexperts_gpt2,gpt2_race,Asian_Americans,toxicity-ratio,0.2397
5,dexperts_gpt2,gpt2_race,African_Americans,regard-positive,0.6233
6,dexperts_gpt2,gpt2_race,African_Americans,regard-negative,0.1410
7,dexperts_gpt2,gpt2_race,African_Americans,regard-neutral,0.0468
8,dexperts_gpt2,gpt2_race,African_Americans,regard-other,0.1889
9,dexperts_gpt2,gpt2_race,African_Americans,toxicity-ratio,0.2279


In [10]:
regardTable = PrettyTable(
    ["Domain", "Model", "Positive", "Negative", " Nuetral"])
toxicityTable = PrettyTable(["Domain", "Model", "Ratio"])


In [11]:
scores = []
groups = df['group'].unique().tolist()

for group in groups:
    # group = 'political_ideology'
    data = df[df['group'] == group]
    model_name = data[data['metric'] ==
                      'regard-positive']['model'].values.tolist()
    subgroup = data[data['metric'] ==
                    'regard-positive']['subgroup'].values.tolist()
    labels = [model_name[i] + '\n' + subgroup[i]
              for i in range(len(model_name))]

    model_name = data[data['metric'] ==
                      'regard-positive']['model'].unique().tolist()
    subgroup = data[data['metric'] ==
                    'regard-positive']['subgroup'].unique().tolist()

    positive_regards = np.array(
        data[data['metric'] == 'regard-positive']['score'].values.tolist())
    negative_regards = np.array(
        data[data['metric'] == 'regard-negative']['score'].values.tolist())
    neutral_regards = np.array(
        data[data['metric'] == 'regard-neutral']['score'].values.tolist())
    toxicity = np.array(
        data[data['metric'] == 'toxicity-ratio']['score'].values.tolist())

    n_subgroups = len(subgroup)
    for i in range(len(model_name)):
        start_ind, end_ind = n_subgroups * i, n_subgroups * (i+1)
        positive = positive_regards[start_ind:end_ind]
        negative = negative_regards[start_ind:end_ind]
        neutral = neutral_regards[start_ind:end_ind]
        toxic = toxicity[start_ind:end_ind]

        regardTable.add_row([group, model_name[i], round(avg_diff(positive), 4), round(
            avg_diff(negative), 4), round(avg_diff(neutral), 4)])
        toxicityTable.add_row(
            [group, model_name[i], round(avg_diff(toxic), 4)])

        scores.append({'model': model_name[i], 'group': group,  'positive': round(avg_diff(positive), 4),
                       'negative': round(avg_diff(negative), 4), 'neutral': round(avg_diff(neutral), 4), 'toxicity_ratio': round(avg_diff(toxic), 4)})


In [12]:
print(regardTable)

+-------------------------+---------------+----------+----------+----------+
|          Domain         |     Model     | Positive | Negative |  Nuetral |
+-------------------------+---------------+----------+----------+----------+
|        gpt2_race        | dexperts_gpt2 |  0.017   |  0.0104  |  0.0032  |
|       gpt2_gender       | dexperts_gpt2 |  0.0342  |  0.0172  |  0.0032  |
| gpt2_religious_ideology | dexperts_gpt2 |  0.0813  |  0.0352  |  0.0131  |
+-------------------------+---------------+----------+----------+----------+


In [13]:
print(toxicityTable)

+-------------------------+---------------+-------+
|          Domain         |     Model     | Ratio |
+-------------------------+---------------+-------+
|        gpt2_race        | dexperts_gpt2 | 0.006 |
|       gpt2_gender       | dexperts_gpt2 | 0.013 |
| gpt2_religious_ideology | dexperts_gpt2 | 0.065 |
+-------------------------+---------------+-------+


In [25]:
#save to csv
df = pd.DataFrame.from_dict(scores)
df.to_csv (r'relative_regard_scores.csv', index = False, header=True)